data의 가장 흥미로운 연구 중 일부는 서로 다른 data sources를 combine 하는데서 나온다. 이 operation들은 두 개의 다른 data를 매우 간단하게 연결하는 것부터 data간 겹치는 부분을 제대로 처리하는 복잡한 database style을 join하고 merge하는 것까지 다양하게 사용될 수 있다. Series와 DataFrame은 이 유형의 operation을 염두에 두고 만들어진 것으로, Pandas는 이러한 유형의 data wrangling(원자료(raw data)를 또 다른 형태로 전환하거나 mapping하는 과정을 말하는 것으로, data munging이라고도 한다.)을 빠르고 간단하게 할 수 있는 function과 method를 제공한다.

여기서는 pd.concat function을 이용한 Series와 DataFrame의 간단한 연결에 대해 알아볼 것이며, 여기서 Pandas에 구현된 더 복잡한 in-memory merge와 join에 대해 자세히 살펴보겠다.

먼저 standard import를 한다.

In [1]:
import pandas as pd
import numpy as np

편의상 앞으로 사용할 특정 형태의 DataFrame을 create하는 function을 다음과 같이 정의하겠다.

In [2]:
def make_df(cols, ind) :
    '''Quickly make a DataFrame'''    
    data = {c : [str(c) + str(i) for i in ind]
           for c in cols}
    return pd.DataFrame (data, ind)

# example DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


# 1. Recall: concatenation of NumPy Arrays(복습 : NumPy 배열 연결)

Series와 DataFrame object의 연결은 48쪽 'The Basics of NumPy Arrays'(NumPy 배열의 기초)에서 살펴본 np.concatenate function을 사용하는 NumPy array concatenation과 매우 유사하다. 이 function을 이용하면 두 개 이상의 array의 contents를 하나의 array로 combine할 수 있다는 점을 기억하자.

In [3]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

첫 번째 parameter는 연결할 array의 list나 tuple이다. 게다가 axis keyword를 사용해 결과를 어느 axis에 따라 연결할 것인지 지정할 수 있다.

In [4]:
x = [[1, 2],
    [3, 4]]
np.concatenate([x, x], axis = 1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

# 2. Simple Concatenation with pd.concat(pd.concat을 이용한 간단한 연결)

Pandas에는 np.concatenate와 syntax가 매우 비슷하지만 다양한 option을 가진 pd.concat() fucntion이 있다. option에 대해서는 잠시 후 알아보겠다.

In [ ]:
# Pandas 0.19 version에서 pd.concat() function signature
pd.concat(objs, axis = 0, join = 'outer', join_axes=None, ignore_index = False,
         keys = None, levels = None, names = None, verify_integrity = False,
         copy = True)

# axis 는 중심선(축)을 의미하며, axes는 복수의 axis 발음은 같음

np.concatenate()를 array를 간단하게 연결하는 데 사용할 수 있는 것처럼 pd.conacat()은 Series나 DataFrame object를 간단하게 연결할 때 사용할 수 있다.

In [6]:
ser1 = pd.Series(['A', 'B', 'C'], index = [1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index = [4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

pd.concat()을 이용하면 DataFrame 같은 higher-dimesional object를 연결할 수도 있다.

In [7]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
print(df1); print(df2); print(pd.concat([df1, df2]))

    A   B
1  A1  B1
2  A2  B2
    A   B
3  A3  B3
4  A4  B4
    A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4


기본적으로 concatenation은 DataFrame 내에서 row-wise(행 단위, 즉 axis = 0)로 일어난다. np.concatenate처럼 pd.cancat도 어느 aixs를 따라 concatenation할 것인지 지정할 수 있다. 아래 예제를 생각해 보자.

In [8]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
print(df3); print(df4); print(pd.concat([df3, df4], axis = 1))

    A   B
0  A0  B0
1  A1  B1
    C   D
0  C0  D0
1  C1  D1
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1


## 1) Duplicate indices(인덱스 복제)

np.concatenate와 pd.concat의 중요한 차이는 Pandas에서의 연결은 그 결과가 복제된 index를 가지더라도 index를 유지한다는 데 있다! 다음과 같은 간단한 예제를 보자.

In [9]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index # make duplicate indides!
print(x); print(y); print(pd.concat([x, y]))

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3


결과에서 index가 반복되는 것을 주목하자. 이것은 DataFrame 내에서는 유효하지만 결과가 바람직하지 않은 경우가 종종 있다. pd.concat()은 이 문제를 처리하는 몇 가지 방법을 제공한다.


### (1) 반복을 error로 잡아낸다. 

pd.concat()의 결과에서 indices가 겹치지 않는지 간단히 검증하고 싶으면 verify_integerity flag를 지정하면 된다. 이 flag를 True로 설정하면 연결 작업에서 duplicate indices가 있을 때 예외(exception)가 발생한다. 확인을 위해 error를 잡아내고 massage를 출력하는 다음 예제를 보자

In [10]:
try:
    pd.concat([x, y], verify_integrity = True)
except ValueError as e:
    print('ValueError : ', e)

ValueError :  Indexes have overlapping values: [0, 1]


### (2) index를 무시한다. 

index 자체가 중요하지 않은 경우에는 그냥 index를 무시하고 싶을 것이다. ignore_index flag를 사용해 이 option을 지정할 수 있다. 이 flag를 True로 설정하면 concatenation 작업은 결과 Series에 새로운 integer index를 생성한다.

In [12]:
print(x); print(y); print(pd.concat([x, y], ignore_index = True))

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3


### (3) MultiIndex keys를 추가한다. 

또 다른 방법은 data source에 대한 label을 지정하는 데 keys 옵션을 사용하는 것이다. 결과는 그 data를 포함하는 hierarchical Indexed series가 될 것이다.

In [14]:
print(x); print(y); print(pd.concat([x, y], keys = ['x', 'y']))

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3


결과는 multiply indexed DataFrame이며, 147쪽 'hierarchical indexing'에서 살펴본 도구를 사용해 이 data를 관심있는 표현 방식으로 전환할 수 있다.


## 2) Concatenation with join(조인을 이용한 연결)

방금 살펴본 간단한 예제에서는 주로 공유된 column names로 DataFrame을 연결했다. 실무에서는 다른 source에서 가져온 data는 다른 column name sets을 가질 수도 있는데, pd.concat이 이 경우를 위한 몇 가지 option을 제공한다. 다음과 같이 공통적인 column 몇 개(전부 아님!)를 가지고 있는 두 개의 DataFrame을 연결하는 것을 생각해보자.

In [20]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
print(df5); print(df6); print(pd.concat([df5, df6]))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4


채울 값이 없는 항목은 기본적으로 NA value로 채워진다. 이 value를 바꾸려면 여러 옵션 중 하나를 지정하면 된다. 기본적으로 join은 input columns의 합집합(union)(join = 'outer')이지만, join = 'inner'를 사용해 이를 columns의 교집합(intersection)으로 변경할 수 있다.

In [21]:
print(df5); print(df6);
print(pd.concat([df5, df6], join = 'inner'))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4


다른 방식은 index object의 list를 취하는 join_axes argument를 사용해 남은 column의 index를 직접 지정하는 것이다. 이 예제에서는 반환된(returned) columns이 첫 번째 input value의 column과 동일해야한다고 지정할 것이다. 

In [22]:
print(df5); print(df6); print(pd.concat([df5, df6], join_axes = [df5.columns]))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
     A   B   C
1   A1  B1  C1
2   A2  B2  C2
3  NaN  B3  C3
4  NaN  B4  C4


pd.concat function의 option을 조합(combination)하면 두 datasets를 연결할 때 다양한 동작이 가능하다. data에 이러한 tool을 사용할 때는 이 점을 유의해 두자.

## 3) The append() method(append() 매서드)

array를 직접 concatenation하는 것이 매우 일반적이라서 Series와 DataFrame object는 더 적은 keystrokes로 똑같은 작업을 수행할 수 있는 append method를 가지고 있다. 예를 들어, pd.concat([df1, df2])를 호출하지 않고 간단하게 df1.append(df2)를 호출할 수 있다.

In [23]:
print(df1); print(df2); print(df1.append(df2))

    A   B
1  A1  B1
2  A2  B2
    A   B
3  A3  B3
4  A4  B4
    A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4


python list의 append(), extend() method와 달리 Pandas의 append() method는 original object를 변경하지 않는 대신 combined data를 가지는 새로운 object를 만든다는 사실을 유념하자. 이 method역시  새 index와 data buffer를 생성하기 때문에 매우 효율적인 method라고 보기는 어렵다. 따라서 append operation을 여러번 수행할 계획이라면 일반적으로 DataFrame의 list를 만들고 그것들을 concat() function에 한 번에 전달하는 것이 더 바람직하다.

다음 절에서는 다양한 source에서 가져온 data를 combining하는 또 다른 강력한 method인 pd.merge에 구현된 database-style의 merges/joins에 대해 살펴보겠다. concat(), append() 및 관련 기능에 대한 더 많은 정보는 Pandas 문서의 'Merge, Join, and Concatenate' section을 참고하자.